First, import needed modules

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
import statsmodels.formula.api as smf
from stargazer.stargazer import Stargazer
from IPython.core.display import HTML
import os

Initalize variables

In [2]:
path = os.getcwd()
print(path)

/Users/jan/Dropbox/UP_EPQM/2222/MA/powerlinemonsters


Import datasets

In [6]:
data = pd.read_csv(f'{path}/data/ltw_treatment.csv', encoding = 'UTF-8-SIG', sep=',')
#data = data.set_index(['AGS', 'year'])
data.head(10)

,AGS,GEN,year,Land,post_2014,Union,SPD,Grüne,FDP,Linke,...,treated_20,treated_30,treated_50,treated_100,treated_0*post_2014,treated_10*post_2014,treated_20*post_2014,treated_30*post_2014,treated_50*post_2014,treated_100*post_2014
0,3101000,Braunschweig,2008.0,Lower-Saxony,0.0,36.861503,31.245464,11.023763,8.087549,9.549529,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,3101000,Braunschweig,2013.0,Lower-Saxony,0.0,29.666789,33.671272,18.608499,8.250600,4.246117,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3101000,Braunschweig,2017.0,Lower-Saxony,1.0,26.342641,37.604703,11.862375,7.904115,6.741527,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
3,3102000,Salzgitter,2008.0,Lower-Saxony,0.0,37.476063,39.268958,4.291938,5.740138,8.828035,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,3102000,Salzgitter,2013.0,Lower-Saxony,0.0,30.880127,44.075623,9.252751,6.053921,4.135136,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
5,3102000,Salzgitter,2017.0,Lower-Saxony,1.0,24.445995,45.203436,4.266019,5.319891,4.862732,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
6,3103000,Wolfsburg,2008.0,Lower-Saxony,0.0,43.121141,31.515784,6.509262,6.691886,7.863727,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,3103000,Wolfsburg,2013.0,Lower-Saxony,0.0,37.629895,35.812910,10.901908,7.342017,3.210075,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,3103000,Wolfsburg,2017.0,Lower-Saxony,1.0,29.942107,43.211115,5.754535,6.092242,4.525280,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,3151001,Adenbüttel,2008.0,Lower-Saxony,0.0,47.568209,30.960854,6.287070,7.117438,4.863582,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


Statsmodels

DiD for treatment 50 in 2014

In [9]:
parties = ['Union',	'SPD', 'FDP', 'Linke', 'Grüne', 'Andere']
for party in parties:
    locals()[party] = smf.ols(formula=f'{party} ~ treated_50 + post_2014 + treated_50*post_2014 + Land', data=data).fit(cov_type='cluster', cov_kwds={'groups': np.array(data[['AGS', 'year']])}) #locals converts string to variable name
stargazer = Stargazer([Union, SPD, FDP, Linke, Grüne, Andere])
stargazer.title('Effect on Primary Vote, Municipalities within 50 km of a powerline project, treatment year 2013')
stargazer.custom_columns(['Union',	'SPD', 'FDP', 'Linke', 'Grüne', 'Andere'], [1, 1, 1, 1, 1, 1])
stargazer.show_model_numbers(False)
with open(f'{path}/tables/ltw/ltw_treatment50.tex', 'w') as f:
    f.write(stargazer.render_latex())
HTML(stargazer.render_html())